In [3]:
!pip install python-dotenv cython

  Using cached Cython-3.0.11-py2.py3-none-any.whl.metadata (3.2 kB)
Using cached Cython-3.0.11-py2.py3-none-any.whl (1.2 MB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
%load_ext cython

# Must set the env var before the RESTClient class is loaded because they use a default arg initializer.

In [2]:
%load_ext dotenv
%dotenv -o

import os
not not os.environ["POLYGON_API_KEY"]

True

In [4]:
import pyarrow as pa
import pyarrow.csv as pa_csv
from pyarrow import dataset as pads
from pyarrow import compute as pac
from pathlib import Path
import pandas as pd
import os

from zipline_polygon_bundle.config import PolygonConfig

In [5]:
environ = {
    # "POLYGON_DATA_DIR": "../data/polygon",
    "POLYGON_DATA_DIR": "/Volumes/nas20t1/Mirror/files.polygon.io",
    # "POLYGON_AGG_TIME": "minute",
}

config = PolygonConfig(
    environ=environ,
    calendar_name="XNYS",
    # start_session="2020-10-07",
    # end_session="2020-10-15",
    # start_session="2003-10-01",
    start_session="2016-01-05",
    # start_session="2023-01-03",
    end_session="2022-12-30",
    # end_session="2023-12-28",
    # end_session="2020-12-31",
    # end_session="2024-06-30",
)

In [6]:
test_csv_path = Path(config.flat_files_dir) / "us_stocks_sip/trades_v1" / "2020/10/2020-10-08.csv.gz"
print(f"{test_csv_path.exists()=}")
test_csv_path

test_csv_path.exists()=True


PosixPath('/Volumes/nas20t1/Mirror/files.polygon.io/flatfiles/us_stocks_sip/trades_v1/2020/10/2020-10-08.csv.gz')

In [7]:
table = pa_csv.read_csv(test_csv_path)
df = table.to_pandas()
df

,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp
0,A,"14,12,37,41",0,11,52983525037648,1602155658646874112,104.12,295001,1602155658646918400,5,1,0,0
1,A,"14,12,37,41",0,11,52983525037649,1602155661112589568,104.00,295201,1602155661112630272,37,1,0,0
2,A,12,0,11,52983525037650,1602155661113007104,104.00,295301,1602155661113041920,158,1,0,0
3,A,"7,12,37",0,4,71675223290963,1602156721498000000,103.88,636301,1602158543191049728,1,1,12,1602158543190686041
4,A,"12,37",0,8,52983525172821,1602161883608071000,104.00,771101,1602161883608327936,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48420262,ZYXI,"12,41",0,8,408,1602196962599455000,18.51,4207423,1602196962599698841,240,3,0,0
48420263,ZYXI,12,0,4,1046,1602199475132000000,19.05,4211428,1602199475134829248,200,3,12,1602199475134789648
48420264,ZYXI,"14,12,41",0,11,598,1602200844791914496,19.05,4213446,1602200844792282485,100,3,0,0
48420265,ZYXI,"14,12,41",0,11,599,1602200877544689664,19.05,4213521,1602200877545059263,100,3,0,0


https://polygon.io/blog/understanding-trade-eligibility

```
The UTP Sale condition matrix is available on our Conditions and Indicators page. This spec answers most questions asked about the eligibility for trades to update the volume value.

Similar to the CTA matrix, this table outlines the eligibility of each value to be updated based on two sets of guidelines: the Consolidated Processing Guidelines and the Market Center Processing Guidelines.

The Consolidated Processing Guidelines pertain to the consolidated tape, specifically the UTP SIP feed.
The Market Center Processing Guidelines apply to individual participants, such as exchanges.
```

In [8]:
from polygon import RESTClient

client = RESTClient()  # POLYGON_API_KEY environment variable is used
conditions = []
for c in client.list_conditions(asset_class="stocks", limit=1000):
    conditions.append(c)
conditions

[Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=1, legacy=None, name='Acquisition', sip_mapping=SipMapping(CTA=None, OPRA=None, UTP='A'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=True, updates_open_close=True, updates_volume=True), market_center=MarketCenter(updates_high_low=True, updates_open_close=True, updates_volume=True))),
 Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=2, legacy=None, name='Average Price Trade', sip_mapping=SipMapping(CTA='B', OPRA=None, UTP='W'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=False, updates_open_close=False, updates_volume=True), market_center=MarketCenter(updates_high_low=False, updates_open_close=False, updates_volume=True))),
 Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=No

In [9]:
trade_conditions = {condition.id: condition for condition in conditions if "trade" in condition.data_types and not condition.legacy}
trade_conditions

{1: Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=1, legacy=None, name='Acquisition', sip_mapping=SipMapping(CTA=None, OPRA=None, UTP='A'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=True, updates_open_close=True, updates_volume=True), market_center=MarketCenter(updates_high_low=True, updates_open_close=True, updates_volume=True))),
 2: Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=2, legacy=None, name='Average Price Trade', sip_mapping=SipMapping(CTA='B', OPRA=None, UTP='W'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=False, updates_open_close=False, updates_volume=True), market_center=MarketCenter(updates_high_low=False, updates_open_close=False, updates_volume=True))),
 3: Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, ex

In [26]:
type(trade_conditions)

dict

In [9]:
df.iloc[0]

ticker                                     A
conditions                       14,12,37,41
correction                                 0
exchange                                  11
id                            52983525037648
participant_timestamp    1602155658646874112
price                                 104.12
sequence_number                       295001
sip_timestamp            1602155658646918400
size                                       5
tape                                       1
trf_id                                     0
trf_timestamp                              0
Name: 0, dtype: object

In [10]:
df.iloc[0].conditions

'14,12,37,41'

In [18]:
trade_conditions[14].update_rules.consolidated.updates_open_close

True

In [38]:
def get_updates_open_close(condition_id: int):
    return trade_conditions[condition_id].update_rules.consolidated.updates_open_close

In [19]:
[get_updates_open_close(int(id)) for id in df.iloc[0].conditions.split(",") if trade_conditions[int(id)].type == "sale_condition"]

NameError: name 'get_updates_open_close' is not defined

In [10]:
%%cython

tc = trade_conditions

def should_update_open_close(row):
    global tc
    should_update = False
    for id in row.conditions.split(","):
        id = int(id)
        if tc[id].type == "sale_condition":
            should_update = tc[id].update_rules.consolidated.updates_open_close
            if not should_update:
                break
    return should_update


Error compiling Cython file:
------------------------------------------------------------
...

tc = trade_conditions
     ^
------------------------------------------------------------

/Users/jim/.cache/ipython/cython/_cython_magic_92472b0cf2bda1b8c2ac37f2072feed1c629786f.pyx:2:5: undeclared name not builtin: trade_conditions


In [23]:
should_update_open_close(df.iloc[0])

NameError: name 'trade_conditions' is not defined

In [24]:
%%cython

from polygon import RESTClient

client = RESTClient()  # POLYGON_API_KEY environment variable is used
conditions = []
for c in client.list_conditions(asset_class="stocks", limit=1000):
    conditions.append(c)

# trade_conditions = {condition.id: condition for condition in conditions if "trade" in condition.data_types and not condition.legacy}
# Need legacy conditions because they appear sometimes, like 35 (Stock Option Trade) on 2020-10-08.
trade_conditions = {condition.id: condition for condition in conditions if "trade" in condition.data_types}

updates_open_close_map = {}
updates_high_low_map = {}
updates_volume_map = {}


def updates_open_close(conditions):
    if not conditions in updates_open_close_map:
        should_update = True
        if conditions.strip():
            for id in conditions.split(","):
                condition = trade_conditions[int(id)]
                if condition.type == "sale_condition":
                    if not condition.update_rules.consolidated.updates_open_close:
                        should_update = False
                        break
        # Only update cache once.
        updates_open_close_map[conditions] = should_update
    return updates_open_close_map[conditions]


def updates_high_low(conditions):
    if not conditions in updates_high_low_map:
        should_update = True
        if conditions.strip():
            for id in conditions.split(","):
                condition = trade_conditions[int(id)]
                if condition.type == "sale_condition":
                    if not condition.update_rules.consolidated.updates_high_low:
                        should_update = False
                        break
        # Only update cache once.
        updates_high_low_map[conditions] = should_update
    return updates_high_low_map[conditions]


def updates_volume(conditions):
    if not conditions in updates_volume_map:
        should_update = True
        if conditions.strip():
            for id in conditions.split(","):
                condition = trade_conditions[int(id)]
                if condition.type == "sale_condition":
                    if not condition.update_rules.consolidated.updates_volume:
                        should_update = False
                        break
        # Only update cache once.
        updates_volume_map[conditions] = should_update
    return updates_volume_map[conditions]


def open_close_price(row):
    if row.conditions.strip():
        for id in row.conditions.split(","):
            condition = trade_conditions[int(id)]
            if condition.type == "sale_condition":
                if not condition.update_rules.consolidated.updates_open_close:
                    return None
    return row["price"]

Content of stderr:
/Users/jim/.cache/ipython/cython/_cython_magic_c7cba4b2dabc98e5eb8b55fd8d33e6bfe250015f.c:6233:26: warning: code will never be executed [-Wunreachable-code]
 6233 |                 module = PyImport_ImportModuleLevelObject(
      |                          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 warning generated.

In [21]:
conditions_values = df.conditions.unique()
conditions_values.tolist()

['14,12,37,41',
 '12',
 '7,12,37',
 '12,37',
 '17,41',
 '16',
 '37',
 '14,41',
 '',
 '14,37,41',
 '10,2,41',
 '10,37,41',
 '53,52,41',
 '15',
 '8,41',
 '12,22',
 '38,41',
 '20,12,41',
 '14,12,41',
 '12,41',
 '12,37,41',
 '37,41',
 '41',
 '12,2',
 '10,41',
 '17,9,41',
 '17,37,41',
 '13,37',
 '53,35,41',
 '32,37',
 '32',
 '7,41',
 '7,37,41',
 '53,41',
 '8,9,41',
 '13',
 '20,12,37,41',
 '20,37,41',
 '53,37,41',
 '7,37',
 '2',
 '20,41',
 '13,22',
 '29,41',
 '20,37',
 '8,37,41',
 '15,41',
 '38',
 '20,12,37',
 '16,41',
 '22,41',
 '7,12',
 '32,2',
 '20,12,2',
 '32,37,41',
 '10,22,41',
 '14,2,41',
 '20,12,22',
 '10,32,37,41',
 '32,22',
 '13,2',
 '13,37,41',
 '52',
 '32,41',
 '14',
 '12,2,41',
 '7,12,37,41',
 '53,32,52,41',
 '20,12',
 '20,10,2,41',
 '2,41',
 '7,10,37,41',
 '53,12,41',
 '7,13,37',
 '20,53,35,41',
 '7,10,41',
 '35',
 '7,12,2,41',
 '18,41',
 '12,22,41',
 '14,32,41',
 '20,53,52,41',
 '29,13',
 '20,13,41',
 '22',
 '20,32,41',
 '20,14,41',
 '29,37,41',
 '7,12,41',
 '7,32,37',
 '20,2,

In [27]:
%%time
[updates_open_close(condition)for condition in conditions_values.tolist()]

CPU times: user 76 μs, sys: 16 μs, total: 92 μs
Wall time: 95.1 μs


[False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False]

In [28]:
%%time
[updates_open_close(condition)for condition in conditions_values.tolist()]

CPU times: user 14 μs, sys: 1e+03 ns, total: 15 μs
Wall time: 15.7 μs


[False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False]

In [15]:
open_close_price(df.iloc[0])

In [29]:
df["updates_open_close"] = df.apply(lambda row: updates_open_close(row.conditions), axis=1) 
df["updates_high_low"] = df.apply(lambda row: updates_high_low(row.conditions), axis=1)
df["updates_volume"] = df.apply(lambda row: updates_volume(row.conditions), axis=1)
df

,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,open_close_price,updates_open_close,updates_high_low,updates_volume
0,A,"14,12,37,41",0,11,52983525037648,1602155658646874112,104.12,295001,1602155658646918400,5,1,0,0,NaN,False,False,True
1,A,"14,12,37,41",0,11,52983525037649,1602155661112589568,104.00,295201,1602155661112630272,37,1,0,0,NaN,False,False,True
2,A,12,0,11,52983525037650,1602155661113007104,104.00,295301,1602155661113041920,158,1,0,0,NaN,False,False,True
3,A,"7,12,37",0,4,71675223290963,1602156721498000000,103.88,636301,1602158543191049728,1,1,12,1602158543190686041,NaN,False,False,True
4,A,"12,37",0,8,52983525172821,1602161883608071000,104.00,771101,1602161883608327936,1,1,0,0,NaN,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48420262,ZYXI,"12,41",0,8,408,1602196962599455000,18.51,4207423,1602196962599698841,240,3,0,0,NaN,False,False,True
48420263,ZYXI,12,0,4,1046,1602199475132000000,19.05,4211428,1602199475134829248,200,3,12,1602199475134789648,NaN,False,False,True
48420264,ZYXI,"14,12,41",0,11,598,1602200844791914496,19.05,4213446,1602200844792282485,100,3,0,0,NaN,False,False,True
48420265,ZYXI,"14,12,41",0,11,599,1602200877544689664,19.05,4213521,1602200877545059263,100,3,0,0,NaN,False,False,True


In [16]:
df["open_close_price"] = df.apply(open_close_price, axis=1)
df

,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,open_close_price
0,A,"14,12,37,41",0,11,52983525037648,1602155658646874112,104.12,295001,1602155658646918400,5,1,0,0,NaN
1,A,"14,12,37,41",0,11,52983525037649,1602155661112589568,104.00,295201,1602155661112630272,37,1,0,0,NaN
2,A,12,0,11,52983525037650,1602155661113007104,104.00,295301,1602155661113041920,158,1,0,0,NaN
3,A,"7,12,37",0,4,71675223290963,1602156721498000000,103.88,636301,1602158543191049728,1,1,12,1602158543190686041,NaN
4,A,"12,37",0,8,52983525172821,1602161883608071000,104.00,771101,1602161883608327936,1,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48420262,ZYXI,"12,41",0,8,408,1602196962599455000,18.51,4207423,1602196962599698841,240,3,0,0,NaN
48420263,ZYXI,12,0,4,1046,1602199475132000000,19.05,4211428,1602199475134829248,200,3,12,1602199475134789648,NaN
48420264,ZYXI,"14,12,41",0,11,598,1602200844791914496,19.05,4213446,1602200844792282485,100,3,0,0,NaN
48420265,ZYXI,"14,12,41",0,11,599,1602200877544689664,19.05,4213521,1602200877545059263,100,3,0,0,NaN


In [48]:
df

,ticker,conditions,correction,exchange,id,participant_timestamp,price,sequence_number,sip_timestamp,size,tape,trf_id,trf_timestamp,time_interval
0,A,"14,12,37,41",0,11,52983525037648,2020-10-08 11:14:18.646874112+00:00,104.12,295001,1602155658646918400,5,1,0,0,2020-10-08 11:14:00+00:00
1,A,"14,12,37,41",0,11,52983525037649,2020-10-08 11:14:21.112589568+00:00,104.00,295201,1602155661112630272,37,1,0,0,2020-10-08 11:14:00+00:00
2,A,12,0,11,52983525037650,2020-10-08 11:14:21.113007104+00:00,104.00,295301,1602155661113041920,158,1,0,0,2020-10-08 11:14:00+00:00
3,A,"7,12,37",0,4,71675223290963,2020-10-08 11:32:01.498000+00:00,103.88,636301,1602158543191049728,1,1,12,1602158543190686041,2020-10-08 11:32:00+00:00
4,A,"12,37",0,8,52983525172821,2020-10-08 12:58:03.608071+00:00,104.00,771101,1602161883608327936,1,1,0,0,2020-10-08 12:58:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48420262,ZYXI,"12,41",0,8,408,2020-10-08 22:42:42.599455+00:00,18.51,4207423,1602196962599698841,240,3,0,0,2020-10-08 22:42:00+00:00
48420263,ZYXI,12,0,4,1046,2020-10-08 23:24:35.132000+00:00,19.05,4211428,1602199475134829248,200,3,12,1602199475134789648,2020-10-08 23:24:00+00:00
48420264,ZYXI,"14,12,41",0,11,598,2020-10-08 23:47:24.791914496+00:00,19.05,4213446,1602200844792282485,100,3,0,0,2020-10-08 23:47:00+00:00
48420265,ZYXI,"14,12,41",0,11,599,2020-10-08 23:47:57.544689664+00:00,19.05,4213521,1602200877545059263,100,3,0,0,2020-10-08 23:47:00+00:00


In [29]:
for id in df.iloc[0].conditions.split(","):
    print(f"{id=} {trade_conditions[int(id)]=}")

id='14' trade_conditions[int(id)]=Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=14, legacy=None, name='Intermarket Sweep', sip_mapping=SipMapping(CTA='F', OPRA=None, UTP='F'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=True, updates_open_close=True, updates_volume=True), market_center=MarketCenter(updates_high_low=True, updates_open_close=True, updates_volume=True)))
id='12' trade_conditions[int(id)]=Condition(abbreviation=None, asset_class='stocks', data_types=['trade'], description=None, exchange=None, id=12, legacy=None, name='Form T/Extended Hours', sip_mapping=SipMapping(CTA='T', OPRA=None, UTP='T'), type='sale_condition', update_rules=UpdateRules(consolidated=Consolidated(updates_high_low=False, updates_open_close=False, updates_volume=True), market_center=MarketCenter(updates_high_low=False, updates_open_close=False, updates_volume=True)))
id='37' trade_conditions[int(i

In [33]:
set(df["conditions"].values.tolist())

{'',
 '10,2,41',
 '10,22,41',
 '10,32,37,41',
 '10,32,41',
 '10,37,41',
 '10,41',
 '12',
 '12,2',
 '12,2,41',
 '12,22',
 '12,22,41',
 '12,37',
 '12,37,41',
 '12,41',
 '13',
 '13,2',
 '13,2,41',
 '13,22',
 '13,37',
 '13,37,41',
 '13,41',
 '14',
 '14,12,37,41',
 '14,12,41',
 '14,2,41',
 '14,32,41',
 '14,37,41',
 '14,41',
 '15',
 '15,41',
 '16',
 '16,41',
 '17,37,41',
 '17,41',
 '17,9,41',
 '18,41',
 '18,9,41',
 '2',
 '2,41',
 '20,10,2,41',
 '20,10,41',
 '20,12',
 '20,12,2',
 '20,12,22',
 '20,12,37',
 '20,12,37,41',
 '20,12,41',
 '20,13,2',
 '20,13,37,41',
 '20,13,41',
 '20,14,41',
 '20,2,41',
 '20,32,41',
 '20,37',
 '20,37,41',
 '20,41',
 '20,52,41',
 '20,53,35,41',
 '20,53,52,41',
 '22',
 '22,41',
 '29,10,2,41',
 '29,12,41',
 '29,13',
 '29,37,41',
 '29,41',
 '31',
 '32',
 '32,2',
 '32,22',
 '32,37',
 '32,37,41',
 '32,41',
 '35',
 '37',
 '37,41',
 '38',
 '38,41',
 '41',
 '52',
 '53,12,41',
 '53,32,52,41',
 '53,35,41',
 '53,37,41',
 '53,41',
 '53,52,41',
 '7,10,37,41',
 '7,10,41',
 '7,12'

In [35]:
df["conditions"].value_counts()

conditions
            17680822
37          15121404
14,37,41     7204704
14,41        6721614
12,37         377988
              ...   
7,32,37            1
7,12,41            1
12,22,41           1
20,14,41           1
32,2               1
Name: count, Length: 102, dtype: int64


In [11]:
df["participant_timestamp"] = pd.to_datetime(
    df["participant_timestamp"], unit="ns", utc=True
)

In [12]:
df["time_interval"] = df["participant_timestamp"].dt.floor("1min")

In [ ]:
df["open_close_price"] =

In [7]:
aggs = df.groupby(["ticker", "time_interval"]).agg({
    'price': ['first', 'max', 'min', 'last'],  # Open, High, Low, Close prices
    'size': 'sum',  # Volume as the sum of size
})
aggs

price                             size
                                   first     max     min      last    sum
ticker time_interval                                                     
A      2020-10-08 11:14:00+00:00  104.12  104.12  104.00  104.0000    200
       2020-10-08 11:32:00+00:00  103.88  103.88  103.88  103.8800      1
       2020-10-08 12:58:00+00:00  104.00  104.00  104.00  104.0000      1
       2020-10-08 13:08:00+00:00  103.88  103.88  103.88  103.8800      5
       2020-10-08 13:30:00+00:00  104.20  104.48  103.91  104.2899  31794
...                                  ...     ...     ...       ...    ...
ZYXI   2020-10-08 21:58:00+00:00   19.10   19.10   19.10   19.1000      2
       2020-10-08 22:29:00+00:00   19.10   19.13   19.10   19.1300    100
       2020-10-08 22:42:00+00:00   19.01   19.01   18.51   18.5100    400
       2020-10-08 23:24:00+00:00   19.05   19.05   19.05   19.0500    200
       2020-10-08 23:47:00+00:00   19.05   19.05   19.05   19.0500    240

[1875459 rows x 5 columns]